# Debug Build lstm

In [1]:
import os
# Filter out logs (additional) : 0 - all logs, 1 - INFO, 2 - WARNING, 3 - ERROR
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'  # or any {'0', '1', '2'}
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import yaml
import pickle as pkl
import subprocess
import tensorflow as tf
from tensorflow.keras.regularizers import L1L2


from src.data_preparation import load_data
from src.data_preparation.blocking_time_series_split import BlockingTimeSeriesSplit 
from src.data_preparation import mdl_dataset

from src.modelling import mdl_input_dico  # input variables class
from src.modelling import mdl_params  # parameters class
from src.modelling import mdl_history

# from src.modelling import model_cnn
from src.modelling import model_lstm
from src.modelling import super_model_dl

from src.visualization import visualize_pca
from src.visualization import mdl_introspect
from src.visualization import mdl_ypred_PCA

from src.utils import reload_config
from src.utils import tardisml_utils

/LUMI_TYKKY_KyB8utx/env1/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
file_config = '../config/config_default_2023.yaml'
    

In [3]:
rootdir = tardisml_utils.get_rootdir()
# file_config = '../config/data_proc_full.yaml'
conf = reload_config.Config(file_config, rootdir=rootdir, verbose=1)

Config file found: ../config/config_default_2023.yaml
PCA results in: /scratch/project_465000269/edelleo1/Leo/results/pca_i100-550_j150-629
Config file updated 'pca_dir': ../config/config_default_2023.yaml
Results in: /scratch/project_465000269/edelleo1/Leo/results/lstm_230919-162150
Folder created

Subfolder created: /scratch/project_465000269/edelleo1/Leo/results/lstm_230919-162150/ml/
Subfolder created: /scratch/project_465000269/edelleo1/Leo/results/lstm_230919-162150/figures/
Config file updated 'results_dir': ../config/config_default_2023.yaml
Config folders updated.
Config copied to: /scratch/project_465000269/edelleo1/Leo/results/lstm_230919-162150
Config file found: /scratch/project_465000269/edelleo1/Leo/results/lstm_230919-162150/config_default_2023.yaml
Default config file is now the copied following one:
/scratch/project_465000269/edelleo1/Leo/results/lstm_230919-162150/config_default_2023.yaml


In [4]:
# ---------------------------------------------------
#                 Defining parameters
# ---------------------------------------------------

H = [1,8,15,22]


new_hist = mdl_history.History(conf)
new_hist.set_zeros()
new_hist.set_SIT_history('keras', H)

# var_to_keep = ['SITf t+0', 'sisnthick t+0', '2T t+0', '2T t+8', 'MSL t+0', 'TP t+0', 'TP t+8', 'siconc t+0'] #tradi
# var_to_keep = ['SITf t+0', 'sisnthick t+0', '2T t+0', 'MSL t+0', 'TP t+0', 'siconc t+0']
var_to_keep = ['SITf t+0', 'sisnthick t+0', '2T t+0', 'MSL t+0', 'TP t+0', 'siconc t+0', 'SIA t+0']

In [5]:
from src.data_preparation import load_data

In [14]:
from importlib import reload
reload(mdl_dataset)

<module 'src.data_preparation.mdl_dataset' from '/users/edelleo1/tardis/tardis-ml/src/data_preparation/mdl_dataset.py'>

In [15]:
reload(load_data)

<module 'src.data_preparation.load_data' from '/users/edelleo1/tardis/tardis-ml/src/data_preparation/load_data.py'>

In [16]:
# ---------------------------------------------------
#                 Loading data
# ---------------------------------------------------

#ds = mdl_dataset.Dataset(conf, setup='no_bias', history=new_hist, var_to_keep=var_to_keep)
ds = mdl_dataset.Dataset(conf, setup=conf.setup, history=new_hist, var_to_keep=var_to_keep)


Initialisation dataset...
Loading data...
Loading sia...
	Retrieve siconc
	Retrieve sisnthick
	Retrieve zos
	Retrieve vxsi
	Retrieve vysi
New variable selection:
  ['SITf t+0 PC0' 'SITf t+0 PC1' 'SITf t+0 PC2' 'SITf t+0 PC3'
 'SITf t+0 PC4' 'SITf t+0 PC5' 'SITf t+0 PC6' 'SITf t+0 PC7'
 'sisnthick t+0 PC0' 'sisnthick t+0 PC1' 'sisnthick t+0 PC2'
 'sisnthick t+0 PC3' '2T t+0 PC0' '2T t+0 PC1' '2T t+0 PC2' '2T t+0 PC3'
 'MSL t+0 PC0' 'MSL t+0 PC1' 'MSL t+0 PC2' 'MSL t+0 PC3' 'TP t+0 PC0'
 'TP t+0 PC1' 'TP t+0 PC2' 'TP t+0 PC3' 'siconc t+0 PC0' 'siconc t+0 PC1'
 'siconc t+0 PC2' 'siconc t+0 PC3']
Variables updated: do not forget to execute self.format_for_dl() if Deep Learning
Size of the training set:    2612 days
Size of the validation set:   653 days
Size of the test set:        1096 days
Config file updated 'ntrain': /scratch/project_465000269/edelleo1/Leo/results/lstm_230919-162150/config_default_2023.yaml
Config file updated 'nval': /scratch/project_465000269/edelleo1/Leo/results/ls

In [ ]:
# -----------------------------------------------------------

regularizers = [L1L2(l1=0, l2=0)]  # [L1L2(l1=0, l2=0), L1L2(l1=0.01, l2=0), L1L2(l1=0, l2=0.01), L1L2(l1=0.1, l2=0.1)]

n_components = ds.config.n_comp['tp']


ireg = 0
reg = regularizers[ireg]
suffixe = f'_reg{ireg}'

            
# n_output = ds[0]['ytrain'].shape[0]
m4 = model_lstm.ModelLSTM(ds, ds.dataset['Xtrain'].shape[1], ds.dataset['Xtrain'].shape[2], reg=reg, rootdir=ds.config.rootdir, ml_dir=ds.config.ml_dir, fig_dir=ds.config.fig_dir)
m4.compile_models(npca=n_components)
m4.fit_multiple(ds.dataset, suffix=suffixe)
m4.print_histories()

# once fit_multiple has find the optimal parameters,
# we retrain ML algorithm by including the validation period
m4.retrain_wval(ds.dataset)

m4.predict_multiple(ds.dataset)
m4.save_prediction()

# m4.save_model()
m4.save_model_weights()


ofile = f'{m4.type}_ypred.png'
mdl_ypred_PCA.draw(m4, odir=m4.rootdir+m4.fig_dir, savefig=True, showfig=False, ofile=ofile)

input change depending on model !!
